<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/stuff/prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install firebase paho-mqtt nltk ipywidgets --quiet

import json
import time
import re
import threading
import paho.mqtt.client as mqtt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from firebase import firebase as fb
from collections import defaultdict
from nltk.stem import PorterStemmer
!pip install ipywidgets
import ipywidgets as widgets
from IPython.display import display, clear_output

# התחברות לפיירבייס
FIREBASE_URL = 'https://shabloolsearchengine-default-rtdb.asia-southeast1.firebasedatabase.app/'
FBconn = fb.FirebaseApplication(FIREBASE_URL, None)
print('✅ Connected to Firebase')

# הגדרת stemmer ו־stop words
stemmer = PorterStemmer()
custom_stop_words = {
    'a', 'an', 'the', 'and', 'or', 'but',
    'to', 'from', 'of', 'in', 'on', 'at', 'for', 'by', 'with', 'as',
    'this', 'that', 'these', 'those', 'it', 'its',
    'be', 'is', 'are', 'was', 'were', 'has', 'have', 'had',
    'not', 'no', 'yes', 'true', 'false', 'null', 'none'
}


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.7 MB/s eta 0:00:00
✅ Connected to Firebase


In [ ]:
def search_mqtt(query, num_results=5):
    index = FBconn.get('/search_index', None)
    if not index:
        return []

    query_words = [stemmer.stem(w.lower()) for w in re.findall(r'\w+', query)]
    if not query_words:
        return []

    page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})
    for term in query_words:
        if term in index:
            for doc_id in index[term]['DocIDs']:
                page_scores[doc_id]['matches'] += 1
                page_scores[doc_id]['total_freq'] += 1

    ranked = sorted([
        (doc_id, score['matches'], score['total_freq'])
        for doc_id, score in page_scores.items()
    ], key=lambda x: (x[1], x[2]), reverse=True)

    return [
        {'doc_id': doc_id, 'matching_words': matches, 'total_frequency': total_freq}
        for doc_id, matches, total_freq in ranked[:num_results]
    ]


In [ ]:
def process_message(data):
    timestamp = str(int(time.time() * 1000))
    FBconn.put('/sensor_data', timestamp, data)

def on_message(client, userdata, msg):
    try:
        payload = json.loads(msg.payload.decode())
        process_message(payload)
        print(f"✅ Saved sensor data: {payload}")
    except json.JSONDecodeError:
        print("❌ Invalid JSON received")

def start_mqtt():
    client = mqtt.Client()
    client.on_message = on_message
    client.connect("test.mosquitto.org", 1883, 60)
    client.subscribe("braude/D106/indoor")
    client.subscribe("braude/D106/outdoor")
    client.loop_start()
    print("📡 Listening for sensor data (MQTT)...")
    time.sleep(60)
    client.loop_stop()
    print("✅ Finished listening")


In [ ]:
start_mqtt()

<ipython-input-3-ff8b4bb1ff4c>:14: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


📡 Listening for sensor data (MQTT)...
✅ Saved sensor data: {'Humidity': 48.39, 'Temperature': 25.52, 'Pressure': 975.69, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.34, 'Temperature': 25.52, 'Pressure': 975.63, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.38, 'Temperature': 25.5, 'Pressure': 975.62, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.41, 'Temperature': 25.51, 'Pressure': 975.63, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.38, 'Temperature': 25.52, 'Pressure': 975.67, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.39, 'Temperature': 25.5, 'Pressure': 975.64, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.4, 'Temperature': 25.5, 'Pressure': 975.63, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.39, 'Temperature': 25.52, 'Pressure': 975.69, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.38, 'Temperature': 25.52, 'Pressure': 975.57, 'Distance': 320.28}
✅ Saved sensor data: {'Humidity': 48.34

In [ ]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import re

base_url = "https://mqtt.org"
start_page = base_url + "/"

index = defaultdict(set)  # term -> set of page IDs
doc_titles = {}           # page ID -> page title

def clean_and_tokenize(text):
    words = re.findall(r'\b[a-zA-Z]{2,}\b', text.lower())
    return [stemmer.stem(word) for word in words if word not in custom_stop_words]

# שלב 1: שליפת כל הקישורים הפנימיים מהעמוד הראשי
try:
    response = requests.get(start_page)
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a', href=True)

    pages = {}
    page_id = 1
    for link in links:
        href = link['href']
        if href.startswith('/'):  # רק קישורים פנימיים
            full_url = base_url + href
            if full_url not in pages.values():
                pages[f"page{page_id}"] = full_url
                page_id += 1

except Exception as e:
    print(f"❌ Failed to crawl links from {start_page}: {e}")
    pages = {}

# שלב 2: אינדוקס הדפים שנמצאו
for doc_id, url in pages.items():
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text()
        words = clean_and_tokenize(text)

        for word in words:
            index[word].add(doc_id)

        title = soup.title.string.strip() if soup.title and soup.title.string else url
        doc_titles[doc_id] = title

        print(f"✅ Indexed {doc_id}: {title}")
    except Exception as e:
        print(f"❌ Failed to load {url}: {e}")

# המרה ל-list לקראת העלאה
index = {term: list(doc_ids) for term, doc_ids in index.items()}

# שלב 3: העלאה לפיירבייס
FBconn.put('/', 'search_index', index)
FBconn.put('/', 'doc_titles', doc_titles)
print("✅ Crawled pages uploaded to Firebase.")


✅ Indexed page1: Getting started
✅ Indexed page2: MQTT Specification
✅ Indexed page3: Software
✅ Indexed page4: Use Cases
✅ Indexed page5: FAQ
✅ Indexed page6: Use Cases
✅ Indexed page7: Use Cases
✅ Indexed page8: Use Cases
✅ Indexed page9: Use Cases
✅ Indexed page10: Use Cases
✅ Indexed page11: Use Cases
✅ Indexed page12: Legal
✅ Crawled pages uploaded to Firebase.


In [ ]:

firebase_ready_index = {
    term: {"term": term, "DocIDs": docs}
    for term, docs in index.items()
}


FBconn.put('/', 'search_index', firebase_ready_index)
print("✅ Index uploaded to Firebase (single request)")


✅ Index uploaded to Firebase (single request)


In [ ]:
def fetch_index():
    raw = FBconn.get('/search_index', None)
    if not raw:
        return {}
    return {term: data['DocIDs'] for term, data in raw.items() if 'DocIDs' in data}


In [ ]:
def admin_screen():
    clear_output()
    index_data = fetch_index()
    terms = list(index_data.keys())
    dropdown = widgets.Dropdown(options=terms, description='Select Term:')
    output = widgets.Output()

    title_html = """
    <div style="text-align:center; margin-bottom:20px;">
        <h1 style="color:#0066cc;">Shablool Search Engine</h1>
        <h3>🛠️ Admin Panel</h3>
        <p>Select a term to delete from the search index.</p>
        <hr style="width:80%; margin:auto;"/>
    </div>
    """
    display(HTML(title_html))

    def delete_term(b):
        full_data = FBconn.get('/search_index', None)
        for key, val in full_data.items():
            if val.get("term") == dropdown.value:
                FBconn.delete('/search_index', key)
                output.clear_output()
                with output:
                    print(f"✅ Term '{dropdown.value}' deleted.")
                return

    def go_back(b):
        clear_output()
        main_dashboard()

    delete_btn = widgets.Button(description='Delete Term', button_style='danger')
    delete_btn.on_click(delete_term)
    back_btn = widgets.Button(description='↩ Back')
    back_btn.on_click(go_back)

    display(widgets.VBox([dropdown, delete_btn, back_btn, output]))


In [ ]:
def search_screen():
    clear_output()
    search_input = widgets.Text(
        placeholder='Enter term...',
        description='Search:',
        layout=widgets.Layout(width='70%'),
        style={'description_width': 'initial'}
    )
    output = widgets.Output()

    def do_search(b):
        query = search_input.value.strip()
        results = search_mqtt(query)
        doc_titles = FBconn.get('/doc_titles', None) or {}

        output.clear_output()
        with output:
            if not results:
                display(HTML(f"<p style='color:red;'>❌ No results found for: <b>{query}</b></p>"))
            else:
                html = f"<p style='color:green;'>✅ {len(results)} result(s) for <b>{query}</b></p><ul style='line-height:1.8;'>"
                for res in results:
                    doc_id = res['doc_id']
                    title = doc_titles.get(doc_id, 'Unknown Title')
                    html += f"<li><b>{doc_id}</b> — <span style='color:#333;'>{title}</span> — Matches: <b>{res['matching_words']}</b>, Frequency: <b>{res['total_frequency']}</b></li>"
                html += "</ul>"
                display(HTML(html))

    def go_back(b):
        clear_output()
        main_dashboard()

    search_btn = widgets.Button(description="Search", button_style='primary', layout=widgets.Layout(width='30%'))
    back_btn = widgets.Button(description="↩ Back", layout=widgets.Layout(width='30%'))
    search_btn.on_click(do_search)
    back_btn.on_click(go_back)

    title = "<h2 style='color:#0066cc;'>🔍 Shablool Search Engine</h2><hr>"
    display(HTML(title))
    display(widgets.VBox([
        search_input,
        widgets.HBox([search_btn, back_btn]),
        output
    ]))



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML

def stats_screen():
    clear_output()

    title_html = """
    <div style="text-align:center; margin-bottom:20px;">
        <h1 style="color:#0066cc;">Shablool Search Engine</h1>
        <h3>📈 Sensor Statistics</h3>
        <p>Displaying changes in sensor readings over time.</p>
        <hr style="border:1px solid #ccc; width:80%; margin:auto;"/>
    </div>
    """
    display(HTML(title_html))

    raw = FBconn.get('/sensor_data', None)
    if not raw:
        display(HTML("<p style='color:red;'>❌ No sensor data available.</p>"))
        return

    df = pd.DataFrame(raw).T
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.sort_index()

    for col in df.columns:
        try:
            df[col] = df[col].astype(float)
            plt.figure(figsize=(10, 4))
            plt.plot(df.index, df[col], marker='o', linestyle='-', color='orange')
            plt.title(f"{col} Readings Over Time")
            plt.xlabel("Timestamp")
            plt.ylabel(col)
            plt.grid(True)
            plt.tight_layout()
            plt.xticks(rotation=30)
            plt.show()
        except:
            pass  # אם יש עמודה שהיא מחרוזת או שגויה

    back_btn = widgets.Button(description='↩ Back', layout=widgets.Layout(width='20%'))
    back_btn.on_click(lambda b: main_dashboard())
    display(back_btn)


In [ ]:
def bonus_feature():
    clear_output()
    data = fetch_index()
    if not data:
        display(HTML("<p style='color:red;'>❌ No index data available.</p>"))
        return

    most_common = max(data.items(), key=lambda item: len(item[1]))
    term, docs = most_common

    html = f"""
    <h2 style='color:#0066cc;'>🌟 Bonus Feature</h2>
    <p>Most frequent term:</p>
    <h3 style='color:green;'>“{term}”</h3>
    <p>Appears in <b>{len(docs)}</b> pages.</p>
    <hr>
    """
    display(HTML(html))

    back_btn = widgets.Button(description='↩ Back')
    back_btn.on_click(lambda b: main_dashboard())
    display(back_btn)


In [ ]:
def main_dashboard():
    clear_output()

    title_html = """
    <h1 style="text-align:center; color:#0066cc;">Shablool Search Engine</h1>
    <hr>
    <p style="text-align:center;">Choose an action:</p>
    """
    display(HTML(title_html))

    btn_admin = widgets.Button(description="🛠️ Admin", button_style='warning', layout=widgets.Layout(width='200px'))
    btn_search = widgets.Button(description="🔍 Search", button_style='primary', layout=widgets.Layout(width='200px'))
    btn_stats = widgets.Button(description="📊 Stats", button_style='success', layout=widgets.Layout(width='200px'))
    btn_bonus = widgets.Button(description="🌟 Bonus", button_style='info', layout=widgets.Layout(width='200px'))

    btn_admin.on_click(lambda b: admin_screen())
    btn_search.on_click(lambda b: search_screen())
    btn_stats.on_click(lambda b: stats_screen())
    btn_bonus.on_click(lambda b: bonus_feature())

    display(widgets.HBox([btn_admin, btn_search, btn_stats, btn_bonus],
                         layout=widgets.Layout(justify_content='center')))


In [ ]:
main_dashboard()
